In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Sao Filipe,14.8961,-24.4956,79.00,76,43,8.90,CV,2021-10-25 23:35:28
1,1,Faya,18.3851,42.4509,59.11,75,0,2.39,SA,2021-10-25 23:38:23
2,2,Tuatapere,-46.1333,167.6833,59.92,85,88,3.11,NZ,2021-10-25 23:35:30
3,3,Ngerengere,-6.7500,38.1167,70.05,96,30,0.49,TZ,2021-10-25 23:40:13
4,4,Hermanus,-34.4187,19.2345,60.91,84,58,15.35,ZA,2021-10-25 23:39:53


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Sao Filipe,14.8961,-24.4956,79.00,76,43,8.90,CV,2021-10-25 23:35:28
5,5,Butaritari,3.0707,172.7902,81.73,74,8,6.08,KI,2021-10-25 23:40:14
7,7,Georgetown,5.4112,100.3354,80.53,92,40,3.00,MY,2021-10-25 23:40:15
8,8,Cabo San Lucas,22.8909,-109.9124,87.62,57,1,12.66,MX,2021-10-25 23:40:15
18,18,Karratha,-20.7377,116.8463,79.25,35,2,15.93,AU,2021-10-25 23:40:20
19,19,San Patricio,28.0170,-97.5169,88.21,70,95,11.81,US,2021-10-25 23:40:20
25,25,Dillon,34.4166,-79.3712,76.98,88,62,9.06,US,2021-10-25 23:40:24
28,28,Rikitea,-23.1203,-134.9692,75.45,72,69,18.92,PF,2021-10-25 23:40:25
32,32,Ladario,-19.0047,-57.6017,76.42,41,76,2.35,BR,2021-10-25 23:40:27
33,33,San Cristobal,7.7669,-72.2250,75.38,99,91,2.42,VE,2021-10-25 23:40:27


In [10]:
preferred_cities_df.count()

City_ID       164
City          164
Lat           164
Lng           164
Max Temp      164
Humidity      164
Cloudiness    164
Wind Speed    164
Country       164
Date          164
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Sao Filipe,CV,79.00,14.8961,-24.4956,
5,Butaritari,KI,81.73,3.0707,172.7902,
7,Georgetown,MY,80.53,5.4112,100.3354,
8,Cabo San Lucas,MX,87.62,22.8909,-109.9124,
18,Karratha,AU,79.25,-20.7377,116.8463,
19,San Patricio,US,88.21,28.0170,-97.5169,
25,Dillon,US,76.98,34.4166,-79.3712,
28,Rikitea,PF,75.45,-23.1203,-134.9692,
32,Ladario,BR,76.42,-19.0047,-57.6017,
33,San Cristobal,VE,75.38,7.7669,-72.2250,


In [13]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None))

In [ ]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

In [18]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [19]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]